# Reto 2
El Reconocimiento de Entidades Nombradas es la tarea de identificar y categorizar información (entidades) entre categorias predefinidas como pueden ser lugares, personas u organizaciones. </br>
Para este reto, spacy para entrenar con nuestros conjuntos de datos, extraímos datos abiertos del gobierno de la siguiente fuente: https://sjf2.scjn.gob.mx/busqueda-principal-tesis

# Imporatación de paqueterías

Para extraer el texto de PDFs, usamos pymupdf una paquetería especializada en la manipulación de archivos entre ellos PDF

In [1]:
!pip install pymupdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 74.5 MB/s eta 0:00:00


Utilizamos <<spacy['transformers']>> para el entrenamiento de nuestro modelo NER

In [2]:
!pip install 'spacy[transformers]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 14.6 MB/s eta 0:00:00


In [14]:
!python -m spacy download es_core_news_sm

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-18 03:32:10.493288: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-18 03:32:10.493428: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-18 03:32:10.493450: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [3]:
import pandas as pd
import numpy as np
import nltk
import spacy
import fitz
import gensim
import json
from spacy import displacy
from spacy.tokens import DocBin
from tqdm import tqdm

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [23]:
doc_fitz = fitz.open("/content/Tesis2026147.pdf") # usamos .open para leer el documento pdf

In [25]:
text1 = [page.get_text().replace("\n", "") for page in doc_fitz]
text1 = ' '.join(text1)
# text1

train es la variable con la que vamos a entrenar el modelo, como estamos usando CPU para el entrenamiento es un proceso computacional tardado, pero se puede generalizar a n etiquetas.

In [9]:
nlp = spacy.blank("es") # creamos un "pipeline" vacio de el lenguaje especificado, español en este caso
train = [
    ('Tesis    Registro digital: 2026047     COMPETENCIA ORIGINARIA DE LA SUPREMA CORTE DE JUSTICIA DE LA NACIÓN. DEBECONOCER DE UN AMPARO EN REVISIÓN CUANDO EXISTE UN PRECEDENTE VINCULANTEQUE SÓLO OTORGA UNA RESPUESTA PARCIAL A LA MATERIA DEL RECURSO.    Hechos: Una persona moral presentó ante la Comisión Federal para la Protección contra RiesgosSanitarios (COFEPRIS) diversas solicitudes de evaluación sanitaria para la importación de distintosproductos, entre ellos, de aceite de cáñamo con concentración de 55 % (cincuenta y cinco porciento) de cannabidiol (CBD) libre de tetrahidrocannabinol (THC) como insumo para procesosindustriales, la cual fue rechazada, con fundamento en los artículos 234, 235 y 235 Bis de la LeyGeneral de Salud. El Juez de Distrito negó el amparo e inconforme la parte quejosa interpusorecurso de revisión que fue remitido por el Tribunal Colegiado de Circuito del conocimiento a esteAlto Tribunal al observar que los precedentes sobre el tema no resolvían la totalidad de lascuestiones constitucionales subsistentes. Criterio jurídico: La Primera Sala de la Suprema Corte de Justicia de la Nación determina quecuando no exista un precedente vinculante que, en términos del artículo 94, párrafo décimosegundo, de la Constitución General, resuelva la totalidad de las interrogantes constitucionalessubsistentes respecto de una ley federal, debe reservar competencia originaria para que sea laSuprema Corte de Justicia de la Nación la que decida, conforme a la lógica de un sistema deprecedentes, si es necesario explorar los alcances de su criterio, si éste se aplica analógicamente, obien, si debe distinguirse para otorgar una respuesta diferenciada. Justificación: Debe recordarse que de conformidad con el artículo 94 constitucional, las razones quejustifiquen las decisiones contenidas en las sentencias dictadas por las Salas de esta SupremaCorte de Justicia de la Nación por mayoría de cuatro votos serán obligatorias para todas lasautoridades jurisdiccionales de la Federación y de las entidades federativas, en cuyo caso los casosfuturos que versen sobre el mismo punto no actualizarán la competencia originaria de esta Cortepara conocer del recurso de revisión respectivo en el juicio de amparo, sino del Tribunal Colegiadode Circuito correspondiente. Sin embargo, existen ocasiones en que la materia del recurso derevisión no logra resolverse exhaustivamente mediante la aplicación de un precedente, en cuyocaso el Tribunal Colegiado debe reservar competencia a este Alto Tribunal para que evalúe la litisconstitucional con el fin de determinar si subsisten cuestiones inéditas, o bien, si es necesario paraavanzar en la construcción de su doctrina. Al realizarse esta evaluación, no sólo pueden suscitarselos dos escenarios claros, en los cuales o bien el problema de constitucionalidad planteado fueabordado con exhaustividad por un criterio previo o no lo fue, sino también casos en los cuales lalitis subsistente en el recurso de revisión sea abordado parcialmente por ejecutorias pasadas. Enesa tercera categoría de casos, esta Corte debe identificar las cuestiones inéditas y, de considerarloprocedente, asumir su competencia originaria. En su caso, si un criterio sirve para dirigir la soluciónInstancia: Primera SalaUndécima ÉpocaMateria(s): Constitucional,ComúnTesis: 1a./J. 34/2023 (11a.)Fuente: Semanario Judicial de laFederación.Tipo: JurisprudenciaSemanario Judicial de la Federaciónhttp://sjf2.scjn.gob.mx/detalle/tesis/2026047Pág. 1 de 2Fecha de impresión 06/03/2023 de la revisión, ello sería posible por su aplicación analógica, lo que estaría sujeto a unaargumentación dirigida a justificar su extensión, lo que es muy propio del sistema de precedentes,que es la que correspondería al fondo del asunto. PRIMERA SALA.    Amparo en revisión 461/2020. 25 de mayo de 2022. Mayoría de cuatro votos de los Ministros JuanLuis González Alcántara Carrancá, Jorge Mario Pardo Rebolledo, quien formuló voto concurrente yAlfredo Gutiérrez Ortiz Mena, y la Ministra Ana Margarita Ríos Farjat. Disidente: Ministra NormaLucía Piña Hernández, quien formuló voto particular. Ponente: Ministro Alfredo Gutiérrez OrtizMena. Secretario: David García Sarubbi. Tesis de jurisprudencia 34/2023 (11a.). Aprobada por la Primera Sala de este Alto Tribunal, ensesión privada de veintidós de febrero de dos mil veintitrés.    Esta tesis se publicó el viernes 03 de marzo de 2023 a las 10:06 horas en el Semanario Judicial dela Federación y, por ende, se considera de aplicación obligatoria a partir del lunes 06 de marzo de2023, para los efectos previstos en el punto noveno del Acuerdo General Plenario 1/2021.Semanario Judicial de la Federaciónhttp://sjf2.scjn.gob.mx/detalle/tesis/2026047Pág. 2 de 2Fecha de impresión 06/03/2023',
  [(355, 363, 'ORGANIZACION')],)
]


spacy solo acepta documentos docbin, por lo tanto cremos un docbin y cargamos los datos a este documento

In [18]:
db = DocBin()
for text, annotations in tqdm(train):
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./train.spacy")

100%|██████████| 1/1 [00:00<00:00, 147.90it/s]


In [19]:
!python -m spacy init fill-config base_config.cfg config.cfg

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-18 03:34:32.493430: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-18 03:34:32.493544: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-18 03:34:32.493563: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

Entrenamos el modelo con un archivo base_config.cfg, que se puede generar desde la pagina de spacy: https://spacy.io/usage/training#quickstart

In [20]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-18 03:34:39.727146: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-18 03:34:39.727324: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-18 03:34:39.727350: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

# Carga de modelos
Usamos el mejor modelo entrenado, sin embargo, se puede notar que se entrenó con una sola etiqueta, esto lo hice porque el modelo tardaba horas en entrenarse con un conjunto de datos mas grande (incluso tardo 5 minutos con solo una etiqueta); aun así esto ejemplifica bien sobre como se puede hacer tu propio modelo de entidad de relaciones nombradas

In [21]:
nlp1 = spacy.load(r"./output/model-best") 

In [26]:
doc_ner = nlp1("Tesis    Registro digital: 2026147     AUTO DE VINCULACIÓN A PROCESO. CESAN SUS EFECTOS CUANDO ES RECLAMADO ENEL JUICIO DE AMPARO INDIRECTO Y CON POSTERIORIDAD SE APLICA UN CRITERIO DEOPORTUNIDAD QUE PRODUCE EL SOBRESEIMIENTO EN LA CAUSA PENAL.    Hechos: Una persona promovió un juicio de amparo indirecto en el que reclamó el auto devinculación a proceso que pesa en su contra por el delito de deserción y la inconstitucionalidad delartículo 269, fracción V, del Código de Justicia Militar. El Juez de Distrito negó el amparo, por lo quela parte quejosa interpuso recurso de revisión, el cual fue remitido a este alto tribunal para conocerdel reclamo de inconstitucionalidad. Durante la tramitación del recurso, el Ministerio Público aplicóun criterio de oportunidad en el procedimiento ordinario y, como consecuencia, el Juez de Controldecretó el sobreseimiento en la causa penal instruida en su contra. Criterio jurídico: Cuando una persona reclama en amparo indirecto el auto de vinculación a procesoque pesa en su contra y con posterioridad el Ministerio Público aplica un criterio de oportunidad enel proceso que produce el sobreseimiento en la causa penal, ello implica que desaparezcan deforma definitiva los efectos del acto reclamado, por lo que la persona regresa a la situación jurídicaanterior a la violación del acto que reclamó, lo que genera que se actualice la causa deimprocedencia de cesación de efectos. Justificación: El criterio de oportunidad es instituido como una forma de terminación de lainvestigación en el proceso penal acusatorio que consiste en la abstención del Ministerio Público deejercer acción penal en contra de la persona imputada, siempre que se hayan reparado ogarantizado los daños causados a la víctima u ofendido. La aplicación del criterio de oportunidadtrae como consecuencia la determinación de sobreseimiento en la causa penal.Con base en lo anterior, si se promueve un juicio de amparo indirecto en el que se reclama un autode vinculación a proceso y posteriormente el Ministerio Público aplica un criterio de oportunidad apartir del cual se decreta el sobreseimiento en la causa penal, ello trae como consecuencia que sedestruyan de manera definitiva los efectos del acto reclamado y que la parte quejosa regrese alestado en que se encontraba antes de la existencia de la violación que reclamó, lo anterior es así,porque la aplicación de un criterio de oportunidad tiene como consecuencia la extinción de la acciónpenal y, en consecuencia, efectos de una sentencia absolutoria, lo cual actualiza la causa deimprocedencia del juicio de amparo por cesación de efectos prevista en el artículo 61, fracción XXI,de la Ley de Amparo. PRIMERA SALA.    Amparo en revisión 219/2022. Esteban Rodríguez Martínez. 9 de noviembre de 2022. Cinco votosde las Ministras Norma Lucía Piña Hernández y Ana Margarita Ríos Farjat, y los Ministros Juan LuisInstancia: Primera SalaUndécima ÉpocaMateria(s): PenalTesis: 1a./J. 45/2023 (11a.)Fuente: Semanario Judicial de laFederación.Tipo: JurisprudenciaSemanario Judicial de la Federaciónhttp://sjf2.scjn.gob.mx/detalle/tesis/2026147Pág. 1 de 2Fecha de impresión 17/03/2023 González Alcántara Carrancá, quien formuló voto concurrente, Jorge Mario Pardo Rebolledo yAlfredo Gutiérrez Ortiz Mena. Ponente: Ministra Presidenta Ana Margarita Ríos Farjat. Secretario:Edwin Antony Pazol Rodríguez. Tesis de jurisprudencia 45/2023 (11a.). Aprobada por la Primera Sala de este Alto Tribunal, ensesión privada de quince de marzo de dos mil veintitrés.    Esta tesis se publicó el viernes 17 de marzo de 2023 a las 10:20 horas en el Semanario Judicial dela Federación y, por ende, se considera de aplicación obligatoria a partir del día hábil siguiente, 22de marzo de 2023, para los efectos previstos en el punto noveno del Acuerdo General Plenario1/2021.Semanario Judicial de la Federaciónhttp://sjf2.scjn.gob.mx/detalle/tesis/2026147Pág. 2 de 2Fecha de impresión 17/03/2023") # input sample text
doc.ents

(COFEPRIS,)